在所有的机器学习分类算法中，朴素贝叶斯(**各特征独立分布**)和其他绝大多数的分类算法都不同。对于大多数的分类算法，比如决策树,KNN,逻辑回归，支持向量机等，他们都是判别方法，也就是直接学习出特征输出Y和特征X之间的关系，要么是决策函数$Y=f(X)$,要么是条件分布$P(Y|X)$。但是朴素贝叶斯却是生成方法，也就是直接找出特征输出Y和特征X的联合分布$P(X,Y)$,然后用$P(Y|X)=P(X,Y)/P(X)$得出。

贝叶斯算法可以很好的处理有限样本情况下的概率分类问题.

## 通过学习类的先验概率和类条件概率来完成
![微信图片_20200522181618.png](https://img01.sogoucdn.com/app/a/100520146/abb44f2fa394b0ffa42ef7c32010df79)

$$P(A|B)=\frac{P(B|A)\times P(A)}{P(B)}$$

In [29]:
import numpy as np 
import pandas as pd 
x1 = [1,1,1,1,1,2,2,2,2,2,3,3,3,3,3]
x2 = ['S','M','M','S','S','S','M','M','L','L','L','M','M','L','L']
y = [-1,-1,1,1,-1,-1,-1,1,1,1,1,1,1,1,-1]
df = pd.DataFrame({'x1':x1, 'x2':x2, 'y':y})
# df.head()
df 

,x1,x2,y
0,1,S,-1
1,1,M,-1
2,1,M,1
3,1,S,1
4,1,S,-1
5,2,S,-1
6,2,M,-1
7,2,M,1
8,2,L,1
9,2,L,1


In [32]:
X=df[df.columns[:-1]]
Y=df[df.columns[-1]]
X,Y

(    x1 x2
 0    1  S
 1    1  M
 2    1  M
 3    1  S
 4    1  S
 5    2  S
 6    2  M
 7    2  M
 8    2  L
 9    2  L
 10   3  L
 11   3  M
 12   3  M
 13   3  L
 14   3  L,
 0    -1
 1    -1
 2     1
 3     1
 4    -1
 5    -1
 6    -1
 7     1
 8     1
 9     1
 10    1
 11    1
 12    1
 13    1
 14   -1
 Name: y, dtype: int64)

## 计算类先验概率和类条件概率
### 定义朴素贝叶斯训练过程：


In [54]:
def nb_fit(X,Y):
    classes=Y.unique()
    class_count=Y.value_counts() 
    class_prior= class_count/len(Y) # 类先验概率 ,P(A):含义是 A 事件存在的概率；
    prior={} # 类条件概率, P(B|A):含义是在A 的结果下出现 B 条件的概率；
    for col in X.columns:
        for j in classes:
            p_x_y=X[Y==j][col].value_counts()
            for i in p_x_y.index:
                prior[(col,i,j)]=p_x_y[i]/class_count[j]
    return classes,class_prior,prior
nb_fit(X,Y)

(array([-1,  1], dtype=int64),
  1    0.6
 -1    0.4
 Name: y, dtype: float64,
 {('x1', 1, -1): 0.5,
  ('x1', 2, -1): 0.3333333333333333,
  ('x1', 3, -1): 0.16666666666666666,
  ('x1', 3, 1): 0.4444444444444444,
  ('x1', 2, 1): 0.3333333333333333,
  ('x1', 1, 1): 0.2222222222222222,
  ('x2', 'S', -1): 0.5,
  ('x2', 'M', -1): 0.3333333333333333,
  ('x2', 'L', -1): 0.16666666666666666,
  ('x2', 'M', 1): 0.4444444444444444,
  ('x2', 'L', 1): 0.4444444444444444,
  ('x2', 'S', 1): 0.1111111111111111})

## 计算其对应的最大后验概率，然后判断其所属的类别
### 给定测试实例并进行预测：

$$P(y=1|x1=2,x2=M)=\frac{P(x1=2|y=1) \times P(x2=M|y=1) \times P(y=1)}{P(x=1,x2=M)}$$
> 
$$P(y=-1|x1=2,x2=M)=\frac{P(x1=2|y=-1) \times P(x2=M|y=-1) \times P(y=-1)}{P(x=1,x2=M)}$$
>因为分母相同，所以计算时省略分母，比较在X_test条件下y=1 和 y=-1 概率，**相对**其他分类结果的概率大着为其分类结果

In [57]:
X_test = {'x1': 2, 'x2': 'M'}
def predict(X_test):
    res=[] # 存储 后验概率
    for c in classes:
        p_y=class_prior[c]
        p_x_y=1 
        for i in X_test.items():
            p_x_y*=prior.get(tuple(list(i)+[c]),0.0004) # 平滑处理:避免0 概率的发生
        res+=[p_y*p_x_y]
    print(res)
    return classes[np.argmax(res)]
"预测数据类别为:",predict(X_test)

[0.044444444444444446, 0.08888888888888888]


('预测数据类别为:', 1)

In [58]:
a=0.88+0.25
b=0.12+0.75 
(0.88*0.13)/(a/(a+b))

0.20247787610619472